In [1]:
#!/usr/bin/env python3
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import elasticsearch
import os
pd.options.plotting.backend = "plotly"
import os 

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
avg_over_min = 1
days_look_back = 40
en_date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
st_date = (datetime.now() - timedelta(days=days_look_back)).strftime('%Y-%m-%dT%H:%M:%S')

en_date_dt = datetime.now().astimezone(pytz.timezone('America/New_York'))
st_date_dt = (datetime.now() - timedelta(days=days_look_back)).astimezone(pytz.timezone('America/New_York'))

st_date_utc = datetime.strptime(st_date, '%Y-%m-%dT%H:%M:%S').astimezone(pytz.UTC).strftime('%Y-%m-%dT%H:%M:%SZ')
en_date_utc = datetime.strptime(en_date, '%Y-%m-%dT%H:%M:%S').astimezone(pytz.UTC).strftime('%Y-%m-%dT%H:%M:%SZ')


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


https://www.wunderground.com/dashboard/pws/KNYNEWYO1597

** Initializing binary search to find the first date with data **
low is 0 - 2022-04-14
high is 40 - 2022-05-24
mid is 20 - 2022-05-04
----//----
low is 0 - 2022-04-14
high is 19 - 2022-05-03
mid is 9 - 2022-04-23
----//----
low is 0 - 2022-04-14
high is 8 - 2022-04-22
mid is 4 - 2022-04-18
----//----
low is 0 - 2022-04-14
high is 3 - 2022-04-17
mid is 1 - 2022-04-15
----//----
low is 0 - 2022-04-14
high is 0 - 2022-04-14
mid is 0 - 2022-04-14
----//----

First date not found!
🌞 🌨 ⛈ from https://www.wunderground.com/dashboard/pws/KNYNEWYO1597/table/2022-04-14/2022-04-14/daily
Saving 279 rows
🌞 🌨 ⛈ from https://www.wunderground.com/dashboard/pws/KNYNEWYO1597/table/2022-04-15/2022-04-15/daily
Saving 284 rows
🌞 🌨 ⛈ from https://www.wunderground.com/dashboard/pws/KNYNEWYO1597/table/2022-04-16/2022-04-16/daily
Saving 282 rows
🌞 🌨 ⛈ from https://www.wunderground.com/dashboard/pws/KNYNEWYO1597/table/2022-04-17/2022-04-17/daily
Saving 28

0

In [ ]:
filePath = "/home/bhavik/the-weather-scraper/KNYNEWYO1597.csv"
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")
os.system('python3 /home/bhavik/the-weather-scraper/weather_scraper.py')

In [3]:
# Import and format Praxis data

uri = 'https://aws.southcoastscience.com/topicMessages?topic=nyu/brooklyn/loc/3/particulates&' \
'startTime=%s&endTime=%s&checkpoint=**:/%i:00' \
% (st_date_utc, en_date_utc, avg_over_min)
print(uri)
praxis_df = pd.DataFrame([])

while uri != '':
    header = {"authorization": "api-key nyu-brooklyn"}
    response = requests.get(uri, headers=header)
    json = response.json()

    data = {}

    data['ts'] = pd.to_datetime([ele['rec'] for ele in json['Items']]).tz_convert(tz='US/Eastern')

    data['praxis_pm1_vals'] = [ele['val']['pm1'] for ele in json['Items']]
    data['praxis_pm2p5_vals'] = [ele['val']['pm2p5'] for ele in json['Items']]
    data['praxis_pm10_vals'] = [ele['val']['pm10'] for ele in json['Items']]

    data['praxis_pm1_vals_adj'] = [ele['exg']['rn20']['pm1'] for ele in json['Items']]
    data['praxis_pm2p5_vals_adj'] = [ele['exg']['rn20']['pm2p5'] for ele in json['Items']]
    data['praxis_pm10_vals_adj'] = [ele['exg']['rn20']['pm10'] for ele in json['Items']]

#     praxis_df = pd.DataFrame(data).set_index('ts').resample(avg_over).mean()

    if 'next' in json:
        uri = json['next']
    else:
        uri = ''
    praxis_df = pd.concat([praxis_df, pd.DataFrame(data)])
    
    time.sleep(0.5)
praxis_df = praxis_df.set_index('ts').resample('%iT' % avg_over_min).mean()

https://aws.southcoastscience.com/topicMessages?topic=nyu/brooklyn/loc/3/particulates&startTime=2022-04-14T01:41:34Z&endTime=2022-05-24T01:41:34Z&checkpoint=**:/1:00


In [4]:
praxis_df

,praxis_pm1_vals,praxis_pm2p5_vals,praxis_pm10_vals,praxis_pm1_vals_adj,praxis_pm2p5_vals_adj,praxis_pm10_vals_adj
ts,,,,,,
2022-04-13 21:42:00-04:00,3.9,8.1,23.3,12.2,15.0,26.1
2022-04-13 21:43:00-04:00,4.0,9.1,23.7,12.9,17.4,30.0
2022-04-13 21:44:00-04:00,3.9,9.2,23.9,13.1,17.5,30.3
2022-04-13 21:45:00-04:00,4.3,10.2,26.9,14.5,19.6,30.6
2022-04-13 21:46:00-04:00,4.3,10.2,25.2,14.2,18.8,29.3
...,...,...,...,...,...,...
2022-05-23 21:38:00-04:00,0.5,1.5,8.2,2.4,3.9,13.1
2022-05-23 21:39:00-04:00,0.5,1.3,7.8,2.4,4.0,14.8
2022-05-23 21:40:00-04:00,0.6,1.7,7.8,2.5,4.0,13.6


In [20]:
df_weather = pd.read_csv('/home/bhavik/the-weather-scraper/KNYNEWYO1597.csv')
print(df_weather)

             Date      Time  Temperature_C  Dew_Point_C  Humidity_%  Wind  \
0      2022/04/13  12:03 AM          14.78         7.56        62.0  East   
1      2022/04/13  12:09 AM          14.89         7.83        63.0  East   
2      2022/04/13  12:14 AM          14.89         7.78        63.0  East   
3      2022/04/13  12:19 AM          14.78         7.33        61.0  East   
4      2022/04/13  12:24 AM          14.89         7.33        61.0  East   
...           ...       ...            ...          ...         ...   ...   
11545  2022/05/23  09:04 PM          21.28         6.17        38.0   NNW   
11546  2022/05/23  09:09 PM          21.17         6.28        38.0   NNW   
11547  2022/05/23  09:14 PM          21.17         6.44        39.0    NW   
11548  2022/05/23  09:19 PM          21.17         6.56        39.0   NNW   
11549  2022/05/23  09:24 PM          21.06         6.33        39.0   WNW   

       Speed_kmh  Gust_kmh  Pressure_hPa  Precip_Rate_mm  Precip_Accum_mm  

In [21]:
# df_weather['ts'] = pd.to_datetime(df_weather['Date'] + ' ' + df_weather['Time']).dt.tz_localize(tz='US/Eastern')
# df_weather = df_weather.set_index('ts').resample('%iT' % avg_over_min).mean()
# df_weather = df_weather.fillna(df_weather.mean())
df_weather['ts'] =pd.to_datetime(df_weather["Date"] + ' ' + df_weather['Time']).dt.tz_localize(tz='US/Eastern')
df_weather= df_weather.set_index('ts').resample('%iT' % avg_over_min).mean().fillna(df_weather.mean())
df_weather = df_weather.loc[st_date:en_date]


/tmp/ipykernel_16012/1415577293.py:5: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_weather= df_weather.set_index('ts').resample('%iT' % avg_over_min).mean().fillna(df_weather.mean())
/tmp/ipykernel_16012/1415577293.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_weather= df_weather.set_index('ts').resample('%iT' % avg_over_min).mean().fillna(df_weather.mean())


In [22]:
# df_weather.tail() 

df_weather = df_weather[['Temperature_C', 'Humidity_%']]
df_weather

,Temperature_C,Humidity_%
ts,,
2022-04-13 21:42:00-04:00,14.769226,57.774805
2022-04-13 21:43:00-04:00,14.769226,57.774805
2022-04-13 21:44:00-04:00,15.940000,73.000000
2022-04-13 21:45:00-04:00,14.769226,57.774805
2022-04-13 21:46:00-04:00,14.769226,57.774805
...,...,...
2022-05-23 21:20:00-04:00,14.769226,57.774805
2022-05-23 21:21:00-04:00,14.769226,57.774805
2022-05-23 21:22:00-04:00,14.769226,57.774805


In [23]:
praxis_df = praxis_df.loc[str(df_weather.index[0]):str(df_weather.index[-1])]
praxis_df


,praxis_pm1_vals,praxis_pm2p5_vals,praxis_pm10_vals,praxis_pm1_vals_adj,praxis_pm2p5_vals_adj,praxis_pm10_vals_adj
ts,,,,,,
2022-04-13 21:42:00-04:00,3.9,8.1,23.3,12.2,15.0,26.1
2022-04-13 21:43:00-04:00,4.0,9.1,23.7,12.9,17.4,30.0
2022-04-13 21:44:00-04:00,3.9,9.2,23.9,13.1,17.5,30.3
2022-04-13 21:45:00-04:00,4.3,10.2,26.9,14.5,19.6,30.6
2022-04-13 21:46:00-04:00,4.3,10.2,25.2,14.2,18.8,29.3
...,...,...,...,...,...,...
2022-05-23 21:20:00-04:00,0.5,1.3,4.2,2.0,3.6,10.8
2022-05-23 21:21:00-04:00,0.5,1.5,4.3,2.1,3.9,12.8
2022-05-23 21:22:00-04:00,0.5,1.2,3.0,1.9,3.6,10.4


In [3]:
GROUP = time.time()

def scroll(es, index, body, scroll='2m', size=1000, timeout=25, **kw):
    if isinstance(timeout, int):
        timeout = '{}s'.format(int(timeout))
    page = es.search(index=index, body=body, scroll=scroll, size=size, timeout=timeout, **kw)
    scroll_id, hits = page['_scroll_id'], page['hits']['hits']
    while len(hits):
        yield hits
        page = es.scroll(scroll_id=scroll_id, scroll=scroll)
        scroll_id, hits = page['_scroll_id'], page['hits']['hits']
        
def sensor_query(key=None, nodeid=None, start=None, end=None, k_time="time", group=GROUP):
    match = []
    if key and nodeid:
        match.append({"term": {f'{key}.keyword': nodeid}})
    end = end or 'now'
    if start:
        match.append({"range" : {k_time : {"gte" : start, "lte" : end}}})
    elif end:
        match.append({"range" : {k_time : {"lte" : end}}})
    return { "query": { "bool": {"must": match} } } if match else {}

def download_sensor_data(table, key=None, nodeid=None, start=None, end=None, save=True, k_time='time', **kw):
    query = sensor_query(key, nodeid, start, end, k_time=k_time, **kw)
    print(query)
    
    def pull():
        with tqdm(scroll(es, table, query)) as pbar:
            for i, hits in enumerate(pbar):
                hits = [h['_source'] for h in hits]
                times = [h[k_time] for h in hits]
                pbar.write('{}. n hits: {}. {} - {}'.format(i, len(hits), min(times), max(times)))
                for h in hits:
                    yield h
    if not save:
        return list(pull())

    fname = 'data/{}/{}.json'.format(group, nodeid or table)
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    print(f'Pulling node={nodeid} for ({start} -> {end}) ... saving to {fname}')
    with open(fname, 'w') as f:
        for h in pull():
            f.write(json.dumps(h) + '\n')
    print('all done!')
    return fname

In [4]:
from importlib import reload
import settings
reload(settings)
es = elasticsearch.Elasticsearch('https://es.master1.sonycproject.com', http_auth=('elastic', settings.es_password))
ss = download_sensor_data('status', 'fqdn', 'sonycnode-dca632ceb490', start='now-%id' % days_look_back, save=False)

data = {}
data['ts'] = pd.to_datetime([datetime.fromtimestamp(int(ele['aq']['dt'])) for ele in ss]).tz_localize(tz='US/Eastern')
    
data['piera7100_pm1_vals'] = [ele['aq']['PM1.0'] for ele in ss]
data['piera7100_pm2p5_vals'] = [ele['aq']['PM2.5'] for ele in ss]
data['piera7100_pm10_vals'] = [ele['aq']['PM10'] for ele in ss]

piera7100_df = pd.DataFrame(data)
    
piera7100_df = piera7100_df.set_index('ts').resample('%iT' % avg_over_min).mean()
# purple_df = purple_df.set_index('ts').resample('%iT' % avg_over_min).mean()
piera7100_df = piera7100_df.loc[st_date_dt:en_date_dt]

{'query': {'bool': {'must': [{'term': {'fqdn.keyword': 'sonycnode-dca632ceb490'}}, {'range': {'time': {'gte': 'now-40d', 'lte': 'now'}}}]}}}


0it [00:00, ?it/s]

0. n hits: 1000. 2022-04-22T11:08:44.952479 - 2022-04-22T18:32:20.064899
1. n hits: 1000. 2022-04-22T10:32:29.556688 - 2022-04-22T19:32:35.718586
2. n hits: 1000. 2022-04-20T20:47:08.801251 - 2022-04-22T19:58:46.082006
3. n hits: 1000. 2022-04-20T16:30:25.871010 - 2022-04-21T02:52:24.156497
4. n hits: 1000. 2022-04-20T17:57:41.881690 - 2022-04-23T13:15:37.680733
5. n hits: 1000. 2022-04-23T09:48:55.452551 - 2022-04-23T15:08:13.819095
6. n hits: 1000. 2022-04-23T10:22:25.828673 - 2022-04-23T15:52:19.366968
7. n hits: 1000. 2022-04-22T23:47:48.741533 - 2022-04-23T15:52:29.361348
8. n hits: 1000. 2022-04-22T23:48:03.680681 - 2022-04-23T03:59:21.541887
9. n hits: 1000. 2022-04-23T02:11:20.252064 - 2022-04-23T03:59:41.524298
10. n hits: 1000. 2022-04-23T03:20:31.081681 - 2022-04-23T11:50:26.811938
11. n hits: 1000. 2022-04-23T09:49:00.376908 - 2022-04-23T17:46:35.534964
12. n hits: 1000. 2022-04-23T11:50:41.771959 - 2022-04-23T18:06:15.744929
13. n hits: 1000. 2022-04-23T12:16:42.095582 - 2

TypeError: 'NoneType' object is not subscriptable

In [ ]:
piera7100_df

,praxis_pm1_vals,praxis_pm2p5_vals,praxis_pm10_vals,praxis_pm1_vals_adj,praxis_pm2p5_vals_adj,praxis_pm10_vals_adj
ts,,,,,,
2022-04-13 21:42:00-04:00,3.9,8.1,23.3,12.2,15.0,26.1
2022-04-13 21:43:00-04:00,4.0,9.1,23.7,12.9,17.4,30.0
2022-04-13 21:44:00-04:00,3.9,9.2,23.9,13.1,17.5,30.3
2022-04-13 21:45:00-04:00,4.3,10.2,26.9,14.5,19.6,30.6
2022-04-13 21:46:00-04:00,4.3,10.2,25.2,14.2,18.8,29.3
...,...,...,...,...,...,...
2022-05-23 21:20:00-04:00,0.5,1.3,4.2,2.0,3.6,10.8
2022-05-23 21:21:00-04:00,0.5,1.5,4.3,2.1,3.9,12.8
2022-05-23 21:22:00-04:00,0.5,1.2,3.0,1.9,3.6,10.4
